In [1]:
import torch
from torch import nn, optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [12]:
from torchvision import datasets
import torchvision.transforms as transforms

num_workers = 0
batch_size = 64
transform = transforms.Compose([
    transforms.Resize(512),
    transforms.CenterCrop(512),
    transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])

tv_transform = transforms.Compose([
    transforms.Resize(512),
    transforms.CenterCrop(512),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])

train_data = datasets.ImageFolder('data/train', transform=transform)
valid_data = datasets.ImageFolder('data/valid', transform=tv_transform)
test_data = datasets.ImageFolder('data/test', transform=tv_transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

loaders = [train_loader, valid_loader, test_loader]

In [13]:
torch.cuda.empty_cache()

In [4]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 1)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.avgpool = nn.AvgPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(64 * 64 * 32, 512)
        self.fc2 = nn.Linear(512,3)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.avgpool(x)
        x = x.view(-1, 64 * 64 * 32)
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.log_softmax(self.fc2(x), dim=1)
        
        return x

In [6]:
transfer = models.alexnet(pretrained=True)
transfer.classifier[6] = nn.Linear(4096, 3)
transfer.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transfer.parameters(), lr=0.01)
transfer.load_state_dict(torch.load('transfer.pth'))

In [10]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loader, model, optimizer, criterion, use_cuda, save_path):
    valid_loss_min = 1.077
    train_loss = 0
    valid_loss = 0
    for epoch in range(1, n_epochs+1):
        model.train()
        for batch_idx, (data, target) in enumerate(loader[0]):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            if batch_idx == 0:
                print('Started Batch')
            optimizer.zero_grad()       
            output = model(data)        
            loss = criterion(output, target)        
            loss.backward()        
            optimizer.step() 
            torch.cuda.empty_cache() 
            train_loss += loss.item()*data.size(0)
        model.eval()
        for batch_idx, (data, target) in enumerate(loader[1]):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            torch.cuda.empty_cache()
        train_loss = train_loss/len(loader[0].dataset)
        valid_loss = valid_loss/len(loader[1].dataset)
        
        print('Epoch: {} \tTraining Loss: {:.3f} \tValidation Loss: {:.3f}'.format(
            epoch, train_loss, valid_loss))
        
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.3f} --> {:.3f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    return model

In [ ]:
train(200, loaders, transfer, optimizer, criterion, True, 'transfer.pth')

Started Batch
Epoch: 1 	Training Loss: 0.834 	Validation Loss: 1.105
Started Batch
Epoch: 2 	Training Loss: 0.835 	Validation Loss: 1.117
Started Batch
Epoch: 3 	Training Loss: 0.835 	Validation Loss: 1.113
Started Batch
Epoch: 4 	Training Loss: 0.835 	Validation Loss: 1.118
Started Batch
Epoch: 5 	Training Loss: 0.835 	Validation Loss: 1.116
Started Batch
Epoch: 6 	Training Loss: 0.835 	Validation Loss: 1.115
Started Batch
Epoch: 7 	Training Loss: 0.835 	Validation Loss: 1.120
Started Batch
Epoch: 8 	Training Loss: 0.835 	Validation Loss: 1.114
Started Batch
Epoch: 9 	Training Loss: 0.835 	Validation Loss: 1.117
Started Batch
Epoch: 10 	Training Loss: 0.835 	Validation Loss: 1.117
Started Batch
Epoch: 11 	Training Loss: 0.835 	Validation Loss: 1.117
Started Batch
Epoch: 12 	Training Loss: 0.835 	Validation Loss: 1.117
Started Batch
Epoch: 13 	Training Loss: 0.835 	Validation Loss: 1.116
Started Batch
Epoch: 14 	Training Loss: 0.835 	Validation Loss: 1.117
Started Batch
Epoch: 15 	Trai

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
        
    model.eval()
    for batch_idx, (data, target) in enumerate(loaders[2]):
        # move to GPU
        print(batch_idx)
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
test(loaders, model, criterion, True)

In [ ]:
from PIL import Image
class_names = [item[4:].replace("_", " ") for item in train_data.classes]

def softmax(one, two, three):
    return np.exp(one)/(np.exp(one)+np.exp(two)+np.exp(three))
   
def display_image(img_path):
    img = cv2.imread(img_path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.show()

def detect(img_path):
    # load the image and return the predicted breed
    display_image(img_path)
    image = Image.open(img_path).convert('RGB')
    in_transform = transforms.Compose([
                        transforms.Resize(256),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
                        
    
    image = in_transform(image)[:3,:,:].unsqueeze(0)
    image= image.cuda()
    output = model(image)
    index = torch.topk(output,3)
    one = index[0][0][0].item()
    two = index[0][0][1].item()
    three = index[0][0][2].item()
    
    return(str(softmax(one,two,three)*100) + '%')

In [ ]:
model.load_state_dict(torch.load('model.pth'))
print(detect('data/test/seborrheic_keratosis/ISIC_0012136.jpg'))